# Hammond Landforms

In [ ]:
# builtins
import pathlib
from collections.abc import Iterable

# externals
from osgeo import gdal
from scipy.ndimage import minimum_filter, maximum_filter, convolve
from scipy.signal import fftconvolve
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

# locals
from pysegcnn.core.utils import img2np, array_replace
from ai4ebv.core.landforms import HammondLandforms, LandformAggregation, SayreLandforms

In [ ]:
# sample digital elevation model
dem = pathlib.Path('/mnt/CEPH_PROJECTS/AI4EBV/LANDFORMS/Python/SRTM_ALPS_32TPS.tif')
elevation = img2np(dem)
plt.imshow(elevation, vmin=0, vmax=4000, cmap='terrain');

## Definition of landform classes

In [ ]:
hammond_slope_classes = np.array([100, 200, 300, 400])
hammond_relief_classes = np.array([10, 20, 30, 40, 50, 60])
hammond_profile_classes = np.array([0, 1, 2, 3, 4])
hammond_landform_classes = []
for k in HammondLandforms.label_dict().keys():
    hammond_landform_classes.extend(k) if isinstance(k, Iterable) else hammond_landform_classes.append(k)
sayre_landform_classes = np.asarray(list(SayreLandforms.label_dict().keys()))[0:-1]
sayre_landform_classes_labels = np.asarray([l['label'] for l in list(SayreLandforms.label_dict().values())])[0:-1]

In [ ]:
# function to plot classified parameters
def plot_parameter(array, classes, cmap='viridis'):
    classes = np.sort(classes)
    im = plt.imshow(array, vmin=classes[0], vmax=classes[-1], cmap=plt.get_cmap(cmap, len(classes)));
    colors = [im.cmap(im.norm(c)) for c in classes]
    patches = [mpatches.Patch(color=colors[i], label="Class: {l}".format(l=classes[i]) ) for i in range(len(classes))]
    plt.legend(handles=patches, bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.,
               ncol = 1 if len(classes) <= 10 else 3)

## Reference: Hammond Landforms as generated by ArcGIS code

In [ ]:
# Hammond landforms as generated by ArcGIS
hlf_arcgis = img2np(dem.parent.joinpath('SRTM_ALPS_32TPS_HLF_ArcGIS.tif')).astype(np.float32)
plot_parameter(hlf_arcgis, hammond_landform_classes)

## Definition of neighborhood analysis windows

In [ ]:
# generic class for NAW's
class NeighborhoodAnalysisWindow(object):
    
    # supported convolutional modes
    modes = ['sum', 'mean']
    
    def __init__(self, mode='sum'):
        # check if mode is supported
        if mode not in self.modes:
            raise ValueError('Mode "{}" not supported. Available modes are {}.'
                             .format(mode, ','.join(self.modes)))
        self.mode = mode
    
    @property
    def neighborhood(self):
        raise NotImplementedError('Define valid cells for this neighborhood.')
    
    @property
    def kernel(self):
        return (self.neighborhood if self.mode == 'sum' else
                self.neighborhood / np.count_nonzero(self.neighborhood))

In [ ]:
# rectangular NAWs
class RectangularNAW(NeighborhoodAnalysisWindow):
    
    def __init__(self, mode='sum', size=(3, 3)):
        super().__init__(mode)
        
        # size of rectangular neighborhood
        self.height, self.width = size
    
    @property
    def neighborhood(self):
        return np.ones(shape=(self.height, self.width))        

In [ ]:
# ciruclar NAWs
class CircularNAW(NeighborhoodAnalysisWindow):
    
    def __init__(self, mode='sum', radius=3):
        super().__init__(mode)
        
        # radius of circular neighborhood
        self.radius = radius

    @property
    def neighborhood(self):
        # mask of pixels within circular neighborhood
        y, x = np.ogrid[-self.radius:self.radius+1, -self.radius:self.radius+1]
        return (x ** 2 + y ** 2 <= self.radius ** 2).astype(np.float32)

## A: Slope

### A0: Define NAW for slope

In [ ]:
# define neighborhood analysis window for slope
naw_slope = CircularNAW(mode='mean', radius=33)
plt.imshow(naw_slope.neighborhood, vmin=0, vmax=1, cmap='Greys_r');

### A1: Calculate percent slope

In [ ]:
# calculate slope in percent for each pixel
slope = gdal.DEMProcessing('', str(dem), 'slope', format='MEM', computeEdges=True, alg='Horn',
            slopeFormat='percent').ReadAsArray().astype(np.float32)
plt.imshow(slope, vmin=0, vmax=100, cmap='terrain');

### A2: Reclassify to gentle slope

In [ ]:
# percent slope < 8%: 0
# percent slope > 8%: 1
gentle_slope = np.where(slope <= 8, 0, 1).astype(np.float32)
plt.imshow(gentle_slope, vmin=0, vmax=1, cmap=plt.get_cmap('Greys', 2));

### A3: Percent of gentle slope in NAW

In [ ]:
# calculate percentage of gentle slope in NAW: fft-convolution
%time gentle_slope_naw = fftconvolve(gentle_slope, naw_slope.kernel, mode='valid') * 100
gentle_slope_naw = np.pad(gentle_slope_naw, int(naw_slope.kernel.shape[0] / 2), mode='constant', constant_values=np.nan)
gentle_slope_naw = gentle_slope_naw.clip(min=0, max=100)  # clip values to valid range
plt.imshow(gentle_slope_naw, vmin=0, vmax=100);

In [ ]:
# calculate percentage of gentle slope in NAW: direct convolution
# %time gentle_slope_naw_direct = convolve(gentle_slope, naw_slope.kernel, mode='constant', cval=np.nan) * 100
# assert np.allclose(gentle_slope_naw, gentle_slope_naw_fft, equal_nan=True, rtol=1e-3)
# print('Mean difference between direct convolution and FFT-convolution: {:.10f}'
#      .format(np.nanmean(gentle_slope_naw_direct - gentle_slope_naw)))

### A4: Hammond slope classes

In [ ]:
# initialize Hammond slope classes
hammond_slope = np.ones(gentle_slope_naw.shape) * np.nan

In [ ]:
# Hammond slope classes
hammond_slope[gentle_slope_naw <= 20] = 400
hammond_slope[(gentle_slope_naw > 20) & (gentle_slope_naw <= 50)] = 300
hammond_slope[(gentle_slope_naw > 50) & (gentle_slope_naw <= 80)] = 200
hammond_slope[gentle_slope_naw > 80] = 100

In [ ]:
plot_parameter(hammond_slope, hammond_slope_classes)

In [ ]:
# check if the slope classes are valid
assert np.array_equal(np.unique(hammond_slope[~np.isnan(hammond_slope)]), hammond_slope_classes)

## B: Relief

### B0: Define NAW for relief

In [ ]:
# define neighborhood analysis window for slope
naw_relief = CircularNAW(mode='sum', radius=33)
plt.imshow(naw_relief.neighborhood, vmin=0, vmax=1, cmap='Greys_r');

### B1: Calculate maximum elevation within NAW

In [ ]:
# maximum elevation within the NAW footprint
%time max_elevation = maximum_filter(elevation, footprint=naw_relief.neighborhood, mode='constant', cval=np.nan).astype(np.float32)
max_elevation[np.isnan(hammond_slope)] = np.nan
plt.imshow(max_elevation, vmin=0, vmax=3500, cmap='terrain');

### B2: Calculate minimum elevation within NAW

In [ ]:
# minimum elevation within the NAW footprint
%time min_elevation = minimum_filter(elevation, footprint=naw_relief.neighborhood, mode='constant', cval=np.nan).astype(np.float32)
min_elevation[np.isnan(hammond_slope)] = np.nan
plt.imshow(min_elevation, vmin=0, vmax=3500, cmap='terrain');

### B3: Calculate local relief

In [ ]:
# calculate local relief
local_relief = (max_elevation - min_elevation).clip(min=0)  # clip to valid range
plt.imshow(local_relief, vmin=0, vmax=900, cmap='terrain');

### B4: Hammond relief classes

In [ ]:
# initialize Hammond relief classes
hammond_relief = np.ones(local_relief.shape) * np.nan

In [ ]:
# Hammond relief classes
hammond_relief[local_relief <= 30] = 10
hammond_relief[(local_relief > 30) & (local_relief <= 90)] = 20
hammond_relief[(local_relief > 90) & (local_relief <= 150)] = 30
hammond_relief[(local_relief > 150) & (local_relief <= 300)] = 40
hammond_relief[(local_relief > 300) & (local_relief <= 900)] = 50
hammond_relief[local_relief > 900] = 60

In [ ]:
plot_parameter(hammond_relief, hammond_relief_classes)

In [ ]:
# check if the relief classes are valid
assert np.array_equal(np.unique(hammond_relief[~np.isnan(hammond_relief)]), hammond_relief_classes)

## C: Profile

### C0: Define slope NAW for profile parameter

In [ ]:
naw_profile = CircularNAW(mode='sum', radius=33)
plt.imshow(naw_profile.neighborhood, vmin=0, vmax=1, cmap='Greys_r');

### C1: Local point of distinction between lowlands and uplands

In [ ]:
# local point of distinction between lowlands and uplands
c1 = local_relief / 2

### C2: Elevation of local point of distinction

In [ ]:
# elevation of local point of distinction between lowlands and uplands
c2 = c1 + min_elevation

### C3: Surface of lowlands vs. uplands

In [ ]:
# surface of lowlands vs. uplands
c3 = elevation - c2
plt.imshow(c3, vmin=-100, vmax=100, cmap='RdBu_r');

### C4-C5: Classify to lowlands and uplands

In [ ]:
# mask of lowlands and uplands
c4_lowlands = (c3 <= 0).astype(np.float32)
c4_uplands = (c3 > 0).astype(np.float32)
plt.imshow(c4_lowlands, vmin=0, vmax=1, cmap='Greys');

### C6: Gentle slope in lowlands

In [ ]:
c6 = c4_lowlands * gentle_slope
plt.imshow(c6, vmin=0, vmax=1, cmap='Greys');

### C7: Sum of gentle slopes in lowlands

In [ ]:
%time c7 = fftconvolve(c6, naw_profile.kernel, mode='valid')
c7 = np.pad(c7, int(naw_profile.kernel.shape[0] / 2), mode='constant', constant_values=np.nan)

### C8: Sum of gentle slopes

In [ ]:
%time c8 = fftconvolve(gentle_slope, naw_profile.kernel, mode='valid')
c8 = np.pad(c8, int(naw_profile.kernel.shape[0] / 2), mode='constant', constant_values=np.nan)

### C9: Percentage of gentle slope in lowlands

In [ ]:
c9 = c7 / c8

### C10-11: Replace NoData values by zero

In [ ]:
c10 = np.nan_to_num(c9, copy=True, nan=0, posinf=0, neginf=0)

### C12: Isolate gentle slopes in lowlands

In [ ]:
c12 = c10 * c4_lowlands

### C13: Percentage of gentle slopes in lowlands

In [ ]:
%time c13 = fftconvolve(c12, naw_slope.kernel, mode='valid') * 100
c13 = np.pad(c13, int(naw_slope.kernel.shape[0] / 2), mode='constant', constant_values=np.nan)
c13 = c13.clip(min=0, max=100)  # clip values to valid range

### C14: Profile in lowland areas

In [ ]:
c14 = np.ones(c13.shape) * np.nan
c14[c13 < 50] = 0
c14[(c13 > 50) & (c13 <= 75)] = 2
c14[c13 > 75] = 1
plt.imshow(c14, vmin=0, vmax=2, cmap=plt.get_cmap('Greys', 3));

### C16: Gentle slope in uplands

In [ ]:
c16 = c4_uplands * gentle_slope
plt.imshow(c16, vmin=0, vmax=1, cmap='Greys');

### C17: Sum of gentle slopes in uplands

In [ ]:
%time c17 = fftconvolve(c16, naw_profile.kernel, mode='valid')
c17 = np.pad(c17, int(naw_profile.kernel.shape[0] / 2), mode='constant', constant_values=np.nan)

### C18: Percentage of gentle slope in uplands

In [ ]:
c18 = c17 / c8

### C19-C20: Replace NoData values by zero

In [ ]:
c19 = np.nan_to_num(c18, copy=True, nan=0, posinf=0, neginf=0)

### C21: Isolate gentle slopes in uplands

In [ ]:
c21 = c19 * c4_uplands

### C22: Percentage of gentle slopes in uplands

In [ ]:
%time c22 = fftconvolve(c21, naw_slope.kernel, mode='valid') * 100
c22 = np.pad(c22, int(naw_slope.kernel.shape[0] / 2), mode='constant', constant_values=np.nan)
c22 = c22.clip(min=0, max=100)  # clip values to valid range

### C23: Profile in uplands areas

In [ ]:
c23 = np.ones(c22.shape) * np.nan
c23[c22 < 50] = 0
c23[(c22 > 50) & (c22 <= 75)] = 3
c23[c22 > 75] = 4
plt.imshow(c23, vmin=0, vmax=4, cmap=plt.get_cmap('Greys', 3));

### C24: Hammond profile classes

In [ ]:
hammond_profile = c14 + c23

In [ ]:
plot_parameter(hammond_profile, hammond_profile_classes)

In [ ]:
# check if the relief classes are valid
assert np.array_equal(np.unique(hammond_profile[~np.isnan(hammond_profile)]), np.asarray([0, 1, 2, 3, 4]))

## D: Hammond Landforms

In [ ]:
# compute hammond landforms
hammond_landforms = hammond_slope + hammond_relief + hammond_profile

In [ ]:
plot_parameter(hammond_landforms, hammond_landform_classes)

## E: Compare Hammond Landforms: Python vs. ArcGIS

In [ ]:
# mask missing values: values at the borders
hlf_python = hammond_landforms.copy()
hlf_arcgis[np.isnan(hammond_landforms)] = 0
hlf_python[np.isnan(hammond_landforms)] = 0

In [ ]:
# mask where Python version and ArcGis version are different
diff = (hlf_python != hlf_arcgis)
difference = {'black': 'Equal', 'white': 'Not equal'}
plt.imshow(diff, vmin=0, vmax=1, cmap='Greys_r');
bpatches = [mpatches.Patch(color=k, label=v) for k, v in difference.items()]
plt.legend(handles=bpatches, bbox_to_anchor=(1.1, 1), loc=2, borderaxespad=0.);

## F: Landforms after Sayre et al. (2020)

In [ ]:
# landforms: Python version
landforms_python = hammond_landforms.copy()

# replace values not defined by NoData value
landforms_python[(~np.isnan(hammond_landforms)) &
                 (~np.isin(hammond_landforms, hammond_landform_classes))] = SayreLandforms.NoData.id  
landforms_python[np.isnan(hammond_landforms)] = SayreLandforms.NoData.id

# aggregate Hammond Landforms to landforms after Sayre et al. (2020)
landforms_python = array_replace(landforms_python, LandformAggregation.to_numpy())
landforms_python[landforms_python == SayreLandforms.NoData.id] = np.nan

In [ ]:
# landforms: Python version
landforms_arcgis = hlf_arcgis.copy()

# replace values not defined by NoData value
landforms_arcgis[(~np.isnan(hammond_landforms)) &
                 (~np.isin(hammond_landforms, hammond_landform_classes))] = SayreLandforms.NoData.id
landforms_arcgis[np.isnan(hammond_landforms)] = SayreLandforms.NoData.id

# aggregate Hammond Landforms to landforms after Sayre et al. (2020)
landforms_arcgis = array_replace(landforms_arcgis, LandformAggregation.to_numpy())
landforms_arcgis[landforms_arcgis == SayreLandforms.NoData.id] = np.nan

In [ ]:
# plot differences in final aggregated landforms
fig, ax = plt.subplots(1, 3, figsize=(16,9))
classes = np.sort(sayre_landform_classes)

# plot python version, arcgis version, and difference
im1 = ax[0].imshow(landforms_python, vmin=classes[0], vmax=classes[-1], cmap=plt.get_cmap('viridis', len(classes)));
im2 = ax[1].imshow(landforms_arcgis, vmin=classes[0], vmax=classes[-1], cmap=plt.get_cmap('viridis', len(classes)));
im3 = ax[2].imshow(landforms_arcgis != landforms_python, vmin=0, vmax=1, cmap='Greys_r');
ax[0].set_title('Landforms: Python')
ax[1].set_title('Landforms: ArcGIS')
ax[2].set_title('Difference')

# add legends
colors = [im1.cmap(im1.norm(c)) for c in classes]
patches = [mpatches.Patch(color=colors[i], label=l) for i, l in enumerate(sayre_landform_classes_labels)]
ax[1].legend(handles=patches, bbox_to_anchor=(0.5, -0.1), borderaxespad=0., ncol=len(classes));
bpatches = [mpatches.Patch(color=k, label=v) for k, v in difference.items()]
ax[-1].legend(handles=bpatches, bbox_to_anchor=(1.1, 1), loc=2, borderaxespad=0.);